In [1]:
import pandas as pd
import regex as re
import numpy as np
from datetime import datetime as dt
from datetime import timedelta

In [2]:
path = 'C:/Users/glh19/OneDrive/Desktop/Winter_Spring 2022/Tosun-Turgut/data/'

adni1 = pd.read_csv(path + 'adni1_muse_results_with_dates.csv')
adni2 = pd.read_csv(path + 'adni2_muse_results_with_dates.csv')
adni3 = pd.read_csv(path + 'adni3_muse_results_with_dates.csv')

DATADIC = pd.read_csv(path + 'DATADIC.csv')
DXSUM = pd.read_csv(path + 'DXSUM_PDXCONV_ADNIALL.csv')
PTDEMOG = pd.read_csv(path + 'PTDEMOG.csv')
# PTDEMOG = pd.read_csv(path + 'PTDEMOG.csv', index_col=False)

### PTDEMOG data of birth column creation

In [3]:
PTDEMOG.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,PTTLANG,PTPLANG,PTPSPEC,PTCOGBEG,PTMCIBEG,PTADBEG,PTADDX,PTETHCAT,PTRACCAT,update_stamp
0,ADNI1,18,2,107,sc,sc,2005-08-17,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-17 00:00:00.0
1,ADNI1,20,1,10,f,f,2005-08-18,NaN,1,2.0,...,-4.0,-4.0,-4,NaN,NaN,-4.0,NaN,-4.0,-4.0,2005-08-18 00:00:00.0
2,ADNI1,22,3,107,sc,sc,2005-08-18,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,1999.0,NaN,2.0,5.0,2005-08-18 00:00:00.0
3,ADNI1,24,4,10,sc,sc,2005-08-18,NaN,1,1.0,...,2.0,2.0,-4,NaN,NaN,-4.0,NaN,1.0,5.0,2005-08-18 00:00:00.0
4,ADNI1,26,5,107,sc,sc,2005-08-23,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-23 00:00:00.0


In [4]:
# PTDEMOG['PTDOBMM']= PTDEMOG['PTDOBMM'].apply(pd.to_numeric, errors='coerce')
# PTDEMOG['PTDOBMM'] = PTDEMOG['PTDOBMM'].dropna()
# PTDEMOG['PTDOBMM'] = PTDEMOG['PTDOBMM'].reset_index(drop=True)

In [5]:
# PTDEMOG['PTDOBMM'] = PTDEMOG['PTDOBMM'].astype(int)
PTDEMOG['PTDOBMM'] = PTDEMOG['PTDOBMM'].astype("Int64")
PTDEMOG['PTDOBMM']

0        4
1       12
2        5
3        1
4       12
        ..
4763     1
4764     1
4765     6
4766    11
4767     6
Name: PTDOBMM, Length: 4768, dtype: Int64

In [6]:
PTDEMOG['PTDOBYY'] = PTDEMOG['PTDOBYY'].astype("Int64")

In [7]:
PTDEMOG['PTDOBYY']

0       1931
1       1944
2       1924
3       1938
4       1931
        ... 
4763    1973
4764    2022
4765    1953
4766    1956
4767    1960
Name: PTDOBYY, Length: 4768, dtype: Int64

In [8]:
# create DOB colulumn

PTDEMOG['defaultDD'] = 15*np.ones(PTDEMOG.shape[0]).astype(int)
PTDEMOG['defaultDD'] = PTDEMOG['defaultDD'].astype("Int64")
PTDEMOG['defaultDD']

0       15
1       15
2       15
3       15
4       15
        ..
4763    15
4764    15
4765    15
4766    15
4767    15
Name: defaultDD, Length: 4768, dtype: Int64

In [9]:
#columns to make date: from month, day, year:

cols = ["PTDOBMM", "defaultDD", "PTDOBYY"]
PTDEMOG["DOB"] = PTDEMOG[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")


PTDEMOG['DOB']

0        4/15/1931
1       12/15/1944
2        5/15/1924
3        1/15/1938
4       12/15/1931
           ...    
4763     1/15/1973
4764     1/15/2022
4765     6/15/1953
4766    11/15/1956
4767     6/15/1960
Name: DOB, Length: 4768, dtype: object

In [10]:
# PTDEMOG['DOB'].str.split('/')
# +'-'+PTDEMOG['DOB'].str.split('/').str[0]+'-'+PTDEMOG['DOB'].str.split('/').str[1]

In [11]:
PTDEMOG['DOB'] = PTDEMOG['DOB'].str.split('/').str[2]+'-'+PTDEMOG['DOB'].str.split('/').str[0]+'-'+PTDEMOG['DOB'].str.split('/').str[1]
# PTDEMOG['DOB'] = PTDEMOG['DOB'].astype(str) 
PTDEMOG['DOB']

0        1931-4-15
1       1944-12-15
2        1924-5-15
3        1938-1-15
4       1931-12-15
           ...    
4763     1973-1-15
4764     2022-1-15
4765     1953-6-15
4766    1956-11-15
4767     1960-6-15
Name: DOB, Length: 4768, dtype: object

In [12]:
#transform to datetime format:

# PTDEMOG['DOB'] = pd.to_datetime(PTDEMOG['DOB'], infer_datetime_format=True)
# # format='%Y-%m-%d'
# PTDEMOG['DOB']

### DXSUM / DATADIC dataframes prep

In [13]:
DXSUM.head(2)
DATADIC.head(2)

,Phase,FLDNAME,TBLNAME,CRFNAME,TEXT,TYPE,LENGTH,CODE,UNITS
0,ADNI1,ID,AAL,NaN,Record ID,N,38,"""crfname"","""",""indexes"",""adni_aal_idx=TBLID,FLD...",NaN
1,ADNI1,SX,AAL,NaN,NaN,N,10,NaN,NaN


In [14]:
PTDEMOG_DXSUM = PTDEMOG.merge(DXSUM,  how='right')
PTDEMOG_DXSUM

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,DXPCOG,DXPATYP,DXPOTHSP,DXDEP,DXDEPSP,DXOTHDEM,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS
0,ADNI1,2.0,2,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
1,ADNI1,336.0,2,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
2,ADNI1,4.0,3,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
3,ADNI1,338.0,3,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
4,ADNI1,990.0,3,107,m12,m12,2006-09-14,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12889,ADNI3,NaN,4556,41,y4,m114,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12890,ADNI3,NaN,7030,25,sc,sc,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12891,ADNI3,NaN,6025,40,y4,m48,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,1.0,scored 4 on GDS compared to 1 last time,NaN,NaN,NaN,NaN,1.0
12892,ADNI3,NaN,6604,50,y2,m24,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0


In [15]:
# will attempt to slim down datframe if index is not altered:

slim_PTDEMOG_DXSUM = PTDEMOG_DXSUM[['Phase','ID', 'RID','PTDOBMM','PTDOBYY','DXCHANGE','DXCURREN','DIAGNOSIS']]
# slim_PTDEMOG_DXSUM = slim_PTDEMOG_DXSUM.reset_index()
slim_PTDEMOG_DXSUM.head(5)

,Phase,ID,RID,PTDOBMM,PTDOBYY,DXCHANGE,DXCURREN,DIAGNOSIS
0,ADNI1,2.0,2,NaN,NaN,NaN,1.0,NaN
1,ADNI1,336.0,2,NaN,NaN,NaN,1.0,NaN
2,ADNI1,4.0,3,NaN,NaN,NaN,3.0,NaN
3,ADNI1,338.0,3,NaN,NaN,NaN,3.0,NaN
4,ADNI1,990.0,3,NaN,NaN,NaN,3.0,NaN


In [16]:
# #drop ADNIGO rows which originated from the DXSUM portion of dataframe
# slim_PTDEMOG_DXSUM = slim_PTDEMOG_DXSUM[slim_PTDEMOG_DXSUM['Phase'] !='ADNIGO']
# slim_PTDEMOG_DXSUM[slim_PTDEMOG_DXSUM['Phase']=='ADNIGO'].size
# # slim_PTDEMOG_DXSUM

#another copy being used for the age section:
clean_PTDEMOG_DXSUM = PTDEMOG_DXSUM[PTDEMOG_DXSUM['Phase'] !='ADNIGO']
clean_PTDEMOG_DXSUM[clean_PTDEMOG_DXSUM['Phase']=='ADNIGO'].size

0

In [17]:
clean_PTDEMOG_DXSUM

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,DXPCOG,DXPATYP,DXPOTHSP,DXDEP,DXDEPSP,DXOTHDEM,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS
0,ADNI1,2.0,2,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
1,ADNI1,336.0,2,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
2,ADNI1,4.0,3,107,bl,bl,2005-10-01,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN
3,ADNI1,338.0,3,107,m06,m06,2006-04-27,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
4,ADNI1,990.0,3,107,m12,m12,2006-09-14,NaN,NaN,NaN,...,-4.0,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12889,ADNI3,NaN,4556,41,y4,m114,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12890,ADNI3,NaN,7030,25,sc,sc,2021-11-30,2021-11-30,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0
12891,ADNI3,NaN,6025,40,y4,m48,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,1.0,scored 4 on GDS compared to 1 last time,NaN,NaN,NaN,NaN,1.0
12892,ADNI3,NaN,6604,50,y2,m24,2021-12-01,2021-12-01,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0


In [18]:
#new unaltered index copy for age calc below:
clean_PTDEMOG_DXSUM['CHAN_CURR'] = clean_PTDEMOG_DXSUM['DXCHANGE'].fillna(value=clean_PTDEMOG_DXSUM['DXCURREN'])

clean_PTDEMOG_DXSUM['complete_diagnosis'] = clean_PTDEMOG_DXSUM['CHAN_CURR'].fillna(value=clean_PTDEMOG_DXSUM['DIAGNOSIS'])

clean_PTDEMOG_DXSUM['complete_diagnosis']

<ipython-input-18-c318c8dd4098>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_PTDEMOG_DXSUM['CHAN_CURR'] = clean_PTDEMOG_DXSUM['DXCHANGE'].fillna(value=clean_PTDEMOG_DXSUM['DXCURREN'])
<ipython-input-18-c318c8dd4098>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_PTDEMOG_DXSUM['complete_diagnosis'] = clean_PTDEMOG_DXSUM['CHAN_CURR'].fillna(value=clean_PTDEMOG_DXSUM['DIAGNOSIS'])


0        1.0
1        1.0
2        3.0
3        3.0
4        3.0
        ... 
12889    1.0
12890    1.0
12891    1.0
12892    2.0
12893    1.0
Name: complete_diagnosis, Length: 12419, dtype: float64

### ADNI dataframe prep

In [19]:
# Normalize the format of 'examDate' column for each dataframe
adni3['examDate'] = pd.to_datetime(adni3['examDate']).dt.date
adni3['examDate']

0         2017-06-21
1         2017-06-21
2         2017-06-21
3         2017-06-21
4         2017-06-21
             ...    
359467    2021-04-05
359468    2021-04-05
359469    2021-04-05
359470    2021-04-05
359471    2021-04-05
Name: examDate, Length: 359472, dtype: object

In [20]:
adni1['examDate'] = pd.to_datetime(adni1['examDate']).dt.date
# adni1['examDate']
adni2['examDate'] = pd.to_datetime(adni2['examDate']).dt.date
# adni2['examDate']

### attempting icv calculations individually for adni 1, 2, 3 

In [21]:
icv_values_og = (adni1[adni1['roi_name'] == 'icv']['roi_volume'])
adni1['thing'] = (adni1[adni1['roi_name'] == 'icv']['roi_volume'])

adni2['thing'] = (adni2[adni2['roi_name'] == 'icv']['roi_volume'])

adni3['thing'] = (adni3[adni3['roi_name'] == 'icv']['roi_volume'])


# icv_ind1 = icv_values_og.index
# icv_val1 = icv_values_og.values

# icv_ind1[0] # 258
# icv_val1[0] #1816835.0527791975
# icv_ind1
# icv_val1

In [22]:
adni1['thing'] = adni1['thing'].fillna(0)

adni2['thing'] = adni2['thing'].fillna(0)

adni3['thing'] = adni3['thing'].fillna(0)

adni1['thing']

0         0.000000e+00
1         0.000000e+00
2         0.000000e+00
3         0.000000e+00
4         0.000000e+00
              ...     
532140    0.000000e+00
532141    0.000000e+00
532142    0.000000e+00
532143    0.000000e+00
532144    2.086241e+06
Name: thing, Length: 532145, dtype: float64

In [23]:
# adni1['icv_vals']  = [adni1[adni1['roi_name']=='icv']['roi_volume'].values if s == 'icv'
#               else 0 for s in adni1['roi_name']]

# icv_values1 = [adni1[adni1['roi_name']=='icv']['roi_volume'].values if s == 'icv'
#               else 0 for s in adni1['roi_name'].values]


# icv_values2 = (adni2[adni2['roi_name'] == 'icv']['roi_volume'])

# icv_values3 = (adni3[adni3['roi_name'] == 'icv']['roi_volume'])


In [24]:
# print("first val")
# # print(adni1['icv_vals'][532144][2])
# print(icv_values1)
# print("last val")
# # print(adni1['icv_vals'][532144][len(adni1['icv_vals'][532144])-1])

In [25]:
adni1['icv_vals'] = np.ones(len(adni1))
adni2['icv_vals'] = np.ones(len(adni2))
adni3['icv_vals'] = np.ones(len(adni3))

adni1['icv_vals']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
532140    1.0
532141    1.0
532142    1.0
532143    1.0
532144    1.0
Name: icv_vals, Length: 532145, dtype: float64

In [26]:
# len(icv_values_og)

# for x in range(0,len(icv_values_og)-1):
#     print(x)
#     print(icv_ind1[x])
#     print(adni1.loc[icv_ind1[x]]['icv_vals'])
#     print(icv_val1[x])

In [27]:
# adni1[adni1['roi_name'] == 'icv']['roi_volume']

In [28]:
# import copy
# adni1_copy = adni1.copy()
# # copy.deepcopy(adni1)
# icv_val1 = adni1[adni1['roi_name'] == 'icv']['roi_volume'].values
# icv_ind1 = adni1[adni1['roi_name'] == 'icv']['roi_volume'].index

# icv_ind1[0] # 258
# icv_val1[0] #1816835.0527791975

# for x in range(0,len(icv_values_og)-1):
#     adni1_copy['icv_vals'][icv_ind1[x]] = icv_val1[x]

    
    
    #     adni1_copy.loc[icv_ind1[x], 'icv_vals'] =  icv_val1[x]
#     A value is trying to be set on a copy of a slice from a DataFrame

In [29]:
# adni1_copy['icv_vals'][icv_ind1[1]]

In [30]:
# for x in range(0,len(icv_values_og)-1):
#     adni1_copy.loc[icv_ind1[x]]['icv_vals'] =  icv_val1[x]

In [31]:
# adni1['icv_vals']
# adni1_copy['icv_vals']

In [32]:
# adni1[adni1['icv_vals']!=1.0]
# adni1['icv_vals']

In [33]:
# adni1.groupby(['project','code'])['roi_volume'].size()

# adni2.groupby(['project','code'])['roi_volume'].size()

# adni3.groupby(['project','code'])['roi_volume'].size()

In [34]:
adni1['icv_vals'] = adni1['icv_vals'].fillna(0)

adni2['icv_vals'] = adni2['icv_vals'].fillna(0)

adni3['icv_vals'] = adni3['icv_vals'].fillna(0)


In [35]:
adni1['icv_vals']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
532140    1.0
532141    1.0
532142    1.0
532143    1.0
532144    1.0
Name: icv_vals, Length: 532145, dtype: float64

In [36]:
#check for duplicate values 
# adni1[adni1.index.duplicated()]

# adni2[adni2.index.duplicated()]

# adni3[adni3.index.duplicated()]

In [37]:
# adni1.groupby(['code','project'])['icv_vals'].max()

# adni2.groupby(['code','project'])['icv_vals'].max()

# adni3.groupby(['code','project'])['icv_vals'].max()

In [38]:
adni1.groupby('code')['thing'].max()

code
002_S_0295X1a_118671           1.816835e+06
002_S_0295X2a_118692           1.853525e+06
002_S_0295X3a_64025            1.854627e+06
002_S_0295X5a_123685           1.861418e+06
002_S_0295X6a_150177           1.829614e+06
                                   ...     
ADNI1_016_S_1121m24_i135570    1.902749e+06
ADNI1_016_S_1138m00_i90875     1.546320e+06
ADNI1_016_S_1149m00_i86336     1.914072e+06
ADNI1_016_S_1326m06_i78841     2.101722e+06
ADNI1_016_S_1326m12_i102675    2.086241e+06
Name: thing, Length: 3273, dtype: float64

In [39]:
def divide_two_cols(adni):
    adni['%_icv'] = adni['roi_volume'] / adni['thing'].max()

#     adni1['%_icv'] = adni['roi_volume'] / adni['icv_vals'].max()
    return adni['%_icv']

adni1['%_icv'] = adni1.groupby('code').apply(divide_two_cols).tolist()

adni2['%_icv'] = adni2.groupby('code').apply(divide_two_cols).tolist()

adni3['%_icv'] = adni3.groupby('code').apply(divide_two_cols).tolist()

# adni123[adni123['code']=='016_S_1326m12_i102675'][['code','roi_name','roi_volume','%_icv']]


In [40]:
# adni1[((adni1['roi_name']=='icv') | (adni1['roi_name']== 'Left Amygdala'))][['project','code','roi_name','roi_volume','icv_vals','thing','%_icv']]

# adni2[((adni2['roi_name']=='icv') | (adni2['roi_name']== 'Left Amygdala'))][['project','code','roi_name','roi_volume','icv_vals','thing','%_icv']]

adni3[((adni3['roi_name']=='icv') | (adni3['roi_name']== 'Left Amygdala'))][['project','code','roi_name','roi_volume','icv_vals','thing','%_icv']]

#this seems to work :) #

,project,code,roi_name,roi_volume,icv_vals,thing,%_icv
118,ADNI3,ADNI3_002_S_0413y00_i863056,Left Amygdala,1.058531e+03,1.0,0.000000e+00,0.000582
258,ADNI3,ADNI3_002_S_0413y00_i863056,icv,1.819225e+06,1.0,1.819225e+06,1.000000
377,ADNI3,ADNI3_002_S_0413y02_i1221051,Left Amygdala,8.900000e+02,1.0,0.000000e+00,0.000624
517,ADNI3,ADNI3_002_S_0413y02_i1221051,icv,1.425667e+06,1.0,1.425667e+06,1.000000
636,ADNI3,ADNI3_002_S_1155y00_i843510,Left Amygdala,1.094572e+03,1.0,0.000000e+00,0.000662
...,...,...,...,...,...,...,...
359165,ADNI3,ADNI3_941_S_6581y02_i1343754,icv,1.713419e+06,1.0,1.713419e+06,1.000000
359172,ADNI3,ADNI3_941_S_6854y00_i1291638,Left Amygdala,9.040000e+02,1.0,0.000000e+00,0.000551
359318,ADNI3,ADNI3_941_S_6854y00_i1291638,icv,1.641144e+06,1.0,1.641144e+06,1.000000
359325,ADNI3,ADNI3_941_S_6854y01_i1428390,Left Amygdala,8.070000e+02,1.0,0.000000e+00,0.000488


### Merge the 3 adni dataframes

In [41]:
adni12 = pd.concat([adni1,adni2], axis=0)
adni123 = pd.concat([adni12, adni3], axis=0)


#check if all three adni dataframes are included
# adni12[adni12['project']=='ADNI2']
# adni123[adni123['project']=='ADNI1']


,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,thing,icv_vals,%_icv
0,ADNI1,002_S_0295X1a_118671,NaN,1.266543e+06,TOTALBRAIN,701.0,2006-04-18,0.000000e+00,1.0,0.697115
1,ADNI1,002_S_0295X1a_118671,NaN,5.994261e+05,GM,601.0,2006-04-18,0.000000e+00,1.0,0.329929
2,ADNI1,002_S_0295X1a_118671,NaN,6.019957e+05,WM,604.0,2006-04-18,0.000000e+00,1.0,0.331343
3,ADNI1,002_S_0295X1a_118671,NaN,2.929851e+05,GM_L,606.0,2006-04-18,0.000000e+00,1.0,0.161261
4,ADNI1,002_S_0295X1a_118671,NaN,2.935244e+05,WM_L,607.0,2006-04-18,0.000000e+00,1.0,0.161558
...,...,...,...,...,...,...,...,...,...,...
532140,ADNI1,ADNI1_016_S_1326m12_i102675,NaN,3.372269e+03,Right TrIFG triangular part of the inferior fr...,204.0,2008-03-31,0.000000e+00,1.0,0.001616
532141,ADNI1,ADNI1_016_S_1326m12_i102675,NaN,3.107519e+03,Left TrIFG triangular part of the inferior fro...,205.0,2008-03-31,0.000000e+00,1.0,0.001490
532142,ADNI1,ADNI1_016_S_1326m12_i102675,NaN,1.183589e+03,Right TTG transverse temporal gyrus,206.0,2008-03-31,0.000000e+00,1.0,0.000567
532143,ADNI1,ADNI1_016_S_1326m12_i102675,NaN,1.644805e+03,Left TTG transverse temporal gyrus,207.0,2008-03-31,0.000000e+00,1.0,0.000788


In [42]:
adni123

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,thing,icv_vals,%_icv
0,ADNI1,002_S_0295X1a_118671,NaN,1.266543e+06,TOTALBRAIN,701.0,2006-04-18,0.0,1.0,0.697115
1,ADNI1,002_S_0295X1a_118671,NaN,5.994261e+05,GM,601.0,2006-04-18,0.0,1.0,0.329929
2,ADNI1,002_S_0295X1a_118671,NaN,6.019957e+05,WM,604.0,2006-04-18,0.0,1.0,0.331343
3,ADNI1,002_S_0295X1a_118671,NaN,2.929851e+05,GM_L,606.0,2006-04-18,0.0,1.0,0.161261
4,ADNI1,002_S_0295X1a_118671,NaN,2.935244e+05,WM_L,607.0,2006-04-18,0.0,1.0,0.161558
...,...,...,...,...,...,...,...,...,...,...
359467,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,2.764000e+03,Right TrIFG triangular part of the inferior fr...,204.0,2021-04-05,0.0,1.0,0.001673
359468,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,2.573000e+03,Left TrIFG triangular part of the inferior fro...,205.0,2021-04-05,0.0,1.0,0.001558
359469,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,1.004000e+03,Right TTG transverse temporal gyrus,206.0,2021-04-05,0.0,1.0,0.000608
359470,ADNI3,ADNI3_941_S_6854y01_i1428390,NaN,1.129000e+03,Left TTG transverse temporal gyrus,207.0,2021-04-05,0.0,1.0,0.000683


In [43]:
# to remove: uppercase items
adni123.drop(adni123.index[adni123['roi_name'].str.isupper()], inplace=True)

In [44]:
discard = (['Right Cerebellum Exterior', 'Left Cerebellum Exterior',
            'Right Cerebellum White Matter','Left Cerebellum White Matter',
            'frontal lobe WM right','frontal lobe WM left',
            'occipital lobe WM right','occipital lobe WM left',
            'parietal lobe WM right','parietal lobe WM left',
            'temporal lobe WM right','temporal lobe WM left',
            'fornix right','fornix left','corpus callosum'])

# to remove unwanted brain regions
adni123 = adni123[adni123.roi_name.isin(discard) == False]

In [45]:
#remove unwanted ADNI1 partial string from code entries:
adni123['code'] = adni123['code'].str.strip('ADNI1_')
adni123['split_code'] = adni123['code'].str.split('_')


<ipython-input-45-79ed33095aa7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adni123['code'] = adni123['code'].str.strip('ADNI1_')
<ipython-input-45-79ed33095aa7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adni123['split_code'] = adni123['code'].str.split('_')


,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,thing,icv_vals,%_icv,split_code
359467,ADNI3,3_941_S_6854y01_i1428390,NaN,2764.0,Right TrIFG triangular part of the inferior fr...,204.0,2021-04-05,0.0,1.0,0.001673,"[3, 941, S, 6854y01, i1428390]"
359468,ADNI3,3_941_S_6854y01_i1428390,NaN,2573.0,Left TrIFG triangular part of the inferior fro...,205.0,2021-04-05,0.0,1.0,0.001558,"[3, 941, S, 6854y01, i1428390]"
359469,ADNI3,3_941_S_6854y01_i1428390,NaN,1004.0,Right TTG transverse temporal gyrus,206.0,2021-04-05,0.0,1.0,0.000608,"[3, 941, S, 6854y01, i1428390]"
359470,ADNI3,3_941_S_6854y01_i1428390,NaN,1129.0,Left TTG transverse temporal gyrus,207.0,2021-04-05,0.0,1.0,0.000683,"[3, 941, S, 6854y01, i1428390]"
359471,ADNI3,3_941_S_6854y01_i1428390,NaN,1652002.0,icv,NaN,2021-04-05,1652002.0,1.0,1.000000,"[3, 941, S, 6854y01, i1428390]"


In [53]:
adni123.head(5)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,thing,icv_vals,%_icv,split_code,SITEID,RID_VISCODE,IMAGEUID
113,ADNI1,002_S_0295X1a_11867,NaN,1673.583956,3rd Ventricle,4.0,2006-04-18,0.0,1.0,0.000921,"[002, S, 0295X1a, 11867]",002,0295X1a,11867
114,ADNI1,002_S_0295X1a_11867,NaN,1684.264134,4th Ventricle,11.0,2006-04-18,0.0,1.0,0.000927,"[002, S, 0295X1a, 11867]",002,0295X1a,11867
115,ADNI1,002_S_0295X1a_11867,NaN,395.166601,Right Accumbens Area,23.0,2006-04-18,0.0,1.0,0.000218,"[002, S, 0295X1a, 11867]",002,0295X1a,11867
116,ADNI1,002_S_0295X1a_11867,NaN,444.295422,Left Accumbens Area,30.0,2006-04-18,0.0,1.0,0.000245,"[002, S, 0295X1a, 11867]",002,0295X1a,11867
117,ADNI1,002_S_0295X1a_11867,NaN,976.168306,Right Amygdala,31.0,2006-04-18,0.0,1.0,0.000537,"[002, S, 0295X1a, 11867]",002,0295X1a,11867


In [51]:
# split up code into new columns: PTID, RID, unique image #
# example code: 002_S_0295xia_118671

adni123['SITEID'] = adni123['split_code'].str[0]
adni123['RID_VISCODE'] = adni123['split_code'].str[2]
adni123['IMAGEUID'] = adni123['split_code'].str[3]

<ipython-input-51-118689246431>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adni123['SITEID'] = adni123['split_code'].str[0]
<ipython-input-51-118689246431>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adni123['RID_VISCODE'] = adni123['split_code'].str[2]
<ipython-input-51-118689246431>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [52]:
# adni123['IMAGEUID']
adni123['RID_VISCODE']

113       0295X1a
114       0295X1a
115       0295X1a
116       0295X1a
117       0295X1a
           ...   
359467          S
359468          S
359469          S
359470          S
359471          S
Name: RID_VISCODE, Length: 585160, dtype: object

In [49]:
# RID=1326 and VISCODE=m12
rid_pattern = r'(\d\d\d\d)'
adni123['RID_VISCODE'] =  adni123['RID_VISCODE'].str.split(pat=rid_pattern)


<ipython-input-49-28a4bfef2917>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adni123['RID_VISCODE'] =  adni123['RID_VISCODE'].str.split(pat=rid_pattern)


In [50]:
# adni123['RID_VISCODE']

113       [, 0295, X1a]
114       [, 0295, X1a]
115       [, 0295, X1a]
116       [, 0295, X1a]
117       [, 0295, X1a]
              ...      
359467              [S]
359468              [S]
359469              [S]
359470              [S]
359471              [S]
Name: RID_VISCODE, Length: 585160, dtype: object

In [ ]:
# adni123['RID_VISCODE'].head(5)
adni123['RID'] = adni123['RID_VISCODE'].str[1]
adni123['VISCODE'] = adni123['RID_VISCODE'].str[2]

In [ ]:
adni123['examDate']

### Calculate % icv value for adni data 

In [ ]:
# icv_values = (adni123[adni123['roi_name'] == 'icv']['roi_volume'])
# icv_values

In [ ]:
# adni123.groupby(['project','code'])['roi_volume'].size()

In [ ]:
# #check for duplicate values 
# adni123[(adni123.index.duplicated())&(adni123['project']=='ADNI1')]
# adni123[adni123.index.duplicated()]

In [ ]:
# check no icv values should be a value of 0
# adni123[(adni123['roi_volume']==0)&(adni123['roi_name']=='Left vessel')]
# adni123[(adni123['roi_volume']==0)&(adni123['roi_name']=='icv')]

In [ ]:
# adni123['icv_vals'] = adni123[adni123['roi_name']=='icv']['roi_volume']
# adni123['icv_vals']

In [ ]:
# adni123['icv_vals'] = adni123['icv_vals'].fillna(0)
# adni123['icv_vals']

In [ ]:
# this indicates some of my icv_vals are 0, hence when dividing to find
# % icv, I get inf
# def trial_func(adni123):
#     a1 = adni123['roi_volume']
#     a2 = adni123['icv_vals'].max()
#     a3 = adni123['roi_volume'] / adni123['icv_vals'].max()
#     return a2
    

# val = adni123.groupby(['code','project']).apply(trial_func).tolist()
# val

In [ ]:
# adni123.groupby(['code','project'])['icv_vals'].max()

In [ ]:
# def divide_two_cols(adni123):
#     adni123['%_icv'] = adni123['roi_volume'] / adni123['icv_vals'].max()
#     return adni123['%_icv']

# # I might have to groupby 'code' and 'project' b/c the same patient may be in multiple phases with different brain vol measurements 
# # checking if function filtered data properly when computing % icv:

# adni123['%_icv'] = adni123.groupby(['code','project']).apply(divide_two_cols).tolist()
# adni123['%_icv'] = adni123.groupby('code').apply(divide_two_cols).tolist()

# # adni123[adni123['code']=='016_S_1326m12_i102675'][['code','roi_name','roi_volume','%_icv']]

# adni123[((adni123['roi_name']=='icv') | (adni123['roi_name']== 'Left Amygdala'))&(adni123['project']=='ADNI1')][['project','code','roi_name','roi_volume','%_icv']]

In [ ]:
clean_PTDEMOG_DXSUM

### Calculate age once after merge 

In [ ]:
adni_merged = pd.merge(adni123.astype(str), clean_PTDEMOG_DXSUM.astype(str), on='RID')

adni_merged

In [ ]:
# compute age:
adni_merged['age_in_days'] = adni_merged['examDate'] -adni_merged['DOB']

adni_merged['age_in_years'] = adni_merged["age_in_days"] / timedelta(days=365)
